In [1]:
import jax.numpy as jnp
from jax import grad
from jax import jit
from jax import random
from jax import value_and_grad
from jax import vmap
from tensorflow_probability.substrates import jax as tfp
import corner

#pip install tfp-nightly[jax]  #for jax aware tfp

# import tensorflow_probability as tfp
# import tensorflow as tf

from skylens import *

In [2]:
from dask.distributed import Lock

In [3]:
import faulthandler; faulthandler.enable()

In [4]:
import multiprocessing
ncpu=multiprocessing.cpu_count()-1
vmem=psutil.virtual_memory()
mem=str(vmem.total/(1024**3)*0.95)+'GB'

In [5]:
from distributed import LocalCluster
from dask.distributed import Client  # we already had this above
#http://distributed.readthedocs.io/en/latest/_modules/distributed/worker.html
c=LocalCluster(n_workers=1,processes=False,memory_limit=mem,threads_per_worker=ncpu,memory_spill_fraction=.99,
               memory_monitor_interval='2000ms')
client=Client(c)

/verafs/home/sukhdeep/.local/lib/python3.8/site-packages/distributed-2.30.1-py3.8.egg/distributed/node.py:151: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 35219 instead
  warnings.warn(


In [6]:
test_run=True

In [7]:
lock = Lock(name="Why_Camb_Why",client=client)
#We get segmentation fault if camb is not under lock and multiple instances are called (tested on single node).
# Why camb is called, multiple instances run, but they all rely on same underlying fortran objects and there are race conditions.
# I tried pickling camb, but Camb cannot be pickled because it's a fortran wrapper and pickle doesn't really know what to do with it. deepcopy throws same error too.
# deepcopy of Ang_PS doesn't capture it because there is no camb object stored as part of PS. It is simply called.


In [8]:
wigner_files={}
# wigner_files[0]= '/Users/Deep/dask_temp/dask_wig3j_l3500_w2100_0_reorder.zarr'
# wigner_files[2]= '/Users/Deep/dask_temp/dask_wig3j_l3500_w2100_2_reorder.zarr'
wig_home='/verafs/scratch/phy200040p/sukhdeep/physics2/skylens/temp/'
wigner_files[0]= wig_home+'dask_wig3j_l3500_w2100_0_reorder.zarr'
wigner_files[2]= wig_home+'/dask_wig3j_l3500_w2100_2_reorder.zarr'


In [9]:
#setup parameters
bin_cl=True
lmax_cl=2000
lmin_cl=2
l0=np.arange(lmin_cl,lmax_cl)

lmin_cl_Bins=lmin_cl+10
lmax_cl_Bins=lmax_cl-10
Nl_bins=25
l_bins=np.int64(np.logspace(np.log10(lmin_cl_Bins),np.log10(lmax_cl_Bins),Nl_bins+1))
lb=np.sqrt(l_bins[1:]*l_bins[:-1])

l=np.unique(np.int64(np.logspace(np.log10(lmin_cl),np.log10(lmax_cl),Nl_bins*20))) #if we want to use fewer ell
bin_cl=True

In [10]:
nzbins=5

In [11]:
if test_run:
    nzbins=2
    lmax_cl=200
#     memory='20gb'
#     ncpu=4
    nwalkers=10
    nsteps=10


In [12]:
xi_win_approx=True
bin_xi=True
xi_SN_analytical=True
th_min=25/60
th_max=250./60
n_th_bins=20
th_bins=np.logspace(np.log10(th_min),np.log10(th_max),n_th_bins+1)
th=np.logspace(np.log10(th_min),np.log10(th_max),n_th_bins*40)
thb=np.sqrt(th_bins[1:]*th_bins[:-1])

In [13]:
WT_kwargs={'l':l0,'theta':th*d2r,'s1_s2':[(2,2),(2,-2),(0,0),(0,2)]}
WT=wigner_transform(wig_d_taper_order_low=6,wig_d_taper_order_high=8,**WT_kwargs)


Wigner ell max too low for theta_min. Recommendation based on first few zeros of bessel  (2, 2)  : [ 330.68725181  759.06522792 1189.97300751 1621.45237761 2053.14856549]


/verafs/home/sukhdeep/.local/lib/python3.8/site-packages/skylens-0.1-py3.8.egg/skylens/wigner_transform.py:102: RuntimeWarning: divide by zero encountered in true_divide
/verafs/home/sukhdeep/.local/lib/python3.8/site-packages/skylens-0.1-py3.8.egg/skylens/wigner_transform.py:102: RuntimeWarning: invalid value encountered in true_divide
/verafs/home/sukhdeep/.local/lib/python3.8/site-packages/skylens-0.1-py3.8.egg/skylens/wigner_transform.py:102: RuntimeWarning: invalid value encountered in cos


Wigner ell max too low for theta_min. Recommendation based on first few zeros of bessel  (2, -2)  : [1043.47198799 1521.50677255 1976.36566123 2422.36921608 2863.90886073]
Wigner ell max too low for theta_min. Recommendation based on first few zeros of bessel  (0, 0)  : [ 330.68725181  759.06522792 1189.97300751 1621.45237761 2053.14856549]
Wigner ell max too low for theta_min. Recommendation based on first few zeros of bessel  (0, 2)  : [ 706.19875936 1157.4541545  1597.84285866 2034.58941842 2469.65245891]
Scattering WT data 1


In [14]:
do_cov=True

SSV_cov=False
tidal_SSV_cov=False
Tri_cov=True

In [15]:
use_window=False
store_win=False

nside=32
window_lmax=nside

In [16]:
zs_bin=lsst_source_tomo_bins(nbins=nzbins,use_window=use_window,nside=nside)

ns0:  27.0


In [17]:
do_cov=True

In [18]:
SSV_cov=False
bin_cl=True
do_cov=True
Tri_cov=False

In [19]:
do_pseudo_cl=False
do_xi=True

In [20]:
use_binned_l=True
use_binned_theta=use_binned_l

In [21]:
corr_ggl=('galaxy','shear')
corr_gg=('galaxy','galaxy')
corr_ll=('shear','shear')
corrs=[corr_ll,corr_ggl,corr_gg]


In [22]:
power_spectra_kwargs={'pk_lock':lock}

In [23]:
#use all ell
kappa0=Skylens(zs_bins=zs_bin,do_cov=do_cov,bin_cl=bin_cl,l_bins=l_bins,l=l0, zg_bins=zs_bin,
               use_window=use_window,Tri_cov=Tri_cov,
               use_binned_l=False,wigner_files=wigner_files,
               SSV_cov=SSV_cov,tidal_SSV_cov=tidal_SSV_cov,f_sky=0.35,
               store_win=store_win,window_lmax=window_lmax,
               sparse_cov=True,corrs=corrs,
               do_xi=do_xi,bin_xi=bin_xi,theta_bins=th_bins,WT=WT,
                use_binned_theta=False,
                nz_PS=10,do_pseudo_cl=do_pseudo_cl,xi_win_approx=True,
                xi_SN_analytical=xi_SN_analytical,#power_spectra_kwargs=power_spectra_kwargs
               )

power spectra camb_pk
Win gen: step size 227 False True 1998 33.0
Window done. Size: 0.0


/verafs/home/sukhdeep/.local/lib/python3.8/site-packages/skylens-0.1-py3.8.egg/skylens/cov_utils.py:79: RuntimeWarning: invalid value encountered in true_divide


In [24]:
if do_xi:
    print('doing xi')
    xi0G=kappa0.xi_tomo()
    xi_cov=client.compute(xi0G['stack']).result()
    cov_inv=np.linalg.inv(xi_cov['cov'].todense())
    data=xi_cov['xi']
else:
    print('doing cl')
    cl0G=kappa0.cl_tomo()
    cl_cov=client.compute(cl0G['stack']).result()
    cov_inv=np.linalg.inv(cl_cov['cov'].todense())
    data=cl_cov['pcl_b']

doing xi


distributed.worker - WARNING -  Compute Failed
Function:  execute_task
args:      ((<function apply at 0x1469e41fe1f0>, <bound method Covariance_utils.xi_cov of <skylens.cov_utils.Covariance_utils object at 0x1469c8120d60>>, [(<class 'tuple'>, [0, 0, 0, 1])], (<class 'dict'>, [['cov_cl', None], ['cls', (<class 'dict'>, [[13, array([8.06555501e-07, 1.05894202e-06, 1.27702410e-06, ...,
       1.33716681e-08, 1.33626402e-08, 1.33536131e-08])], [24, array([8.06555501e-07, 1.05894202e-06, 1.27702410e-06, ...,
       1.33716681e-08, 1.33626402e-08, 1.33536131e-08])], [14, array([8.06555501e-07, 1.05894202e-06, 1.27702410e-06, ...,
       1.33716681e-08, 1.33626402e-08, 1.33536131e-08])], [23, array([8.06555501e-07, 1.05894202e-06, 1.27702410e-06, ...,
       1.33716681e-08, 1.33626402e-08, 1.33536131e-08])]])], ['s1_s2', (<class 'tuple'>, [0, 0])], ['SN', (<class 'dict'>, [[(<class 'tuple'>, ['galaxy', 'galaxy']), array([[[4.93274276e-09, 0.00000000e+00],
        [0.00000000e+00, 8.59388519e

distributed.worker - WARNING -  Compute Failed
Function:  execute_task
args:      ((<function apply at 0x1469e41fe1f0>, <bound method Covariance_utils.xi_cov of <skylens.cov_utils.Covariance_utils object at 0x1469c8120d60>>, [(<class 'tuple'>, [0, 1, 0, 1])], (<class 'dict'>, [['cov_cl', None], ['cls', (<class 'dict'>, [[13, array([8.06555501e-07, 1.05894202e-06, 1.27702410e-06, ...,
       1.33716681e-08, 1.33626402e-08, 1.33536131e-08])], [24, array([3.26470135e-08, 4.86787844e-08, 5.85202207e-08, ...,
       4.29967075e-10, 4.29672121e-10, 4.29377188e-10])], [14, array([1.99272924e-08, 2.77690215e-08, 3.16494253e-08, ...,
       1.10043657e-10, 1.09962855e-10, 1.09882058e-10])], [23, array([8.06555501e-07, 1.05894202e-06, 1.27702410e-06, ...,
       1.33716681e-08, 1.33626402e-08, 1.33536131e-08])]])], ['s1_s2', (<class 'tuple'>, [2, 2])], ['SN', (<class 'dict'>, [[(<class 'tuple'>, ['galaxy', 'galaxy']), array([[[4.93274276e-09, 0.00000000e+00],
        [0.00000000e+00, 8.59388519e

distributed.worker - WARNING -  Compute Failed
Function:  execute_task
args:      ((<function apply at 0x1469e41fe1f0>, <bound method Covariance_utils.xi_cov of <skylens.cov_utils.Covariance_utils object at 0x1469c8120d60>>, [(<class 'tuple'>, [0, 1, 0, 0])], (<class 'dict'>, [['cov_cl', None], ['cls', (<class 'dict'>, [[13, array([8.06555501e-07, 1.05894202e-06, 1.27702410e-06, ...,
       1.33716681e-08, 1.33626402e-08, 1.33536131e-08])], [24, array([1.99272924e-08, 2.77690215e-08, 3.16494253e-08, ...,
       1.10043657e-10, 1.09962855e-10, 1.09882058e-10])], [14, array([1.99272924e-08, 2.77690215e-08, 3.16494253e-08, ...,
       1.10043657e-10, 1.09962855e-10, 1.09882058e-10])], [23, array([8.06555501e-07, 1.05894202e-06, 1.27702410e-06, ...,
       1.33716681e-08, 1.33626402e-08, 1.33536131e-08])]])], ['s1_s2', (<class 'tuple'>, [2, 2])], ['SN', (<class 'dict'>, [[(<class 'tuple'>, ['galaxy', 'galaxy']), array([[[4.93274276e-09, 0.00000000e+00],
        [0.00000000e+00, 8.59388519e

distributed.worker - WARNING -  Compute Failed
Function:  execute_task
args:      ((<function apply at 0x1469e41fe1f0>, <bound method Covariance_utils.xi_cov of <skylens.cov_utils.Covariance_utils object at 0x1469c8120d60>>, [(<class 'tuple'>, [0, 0, 1, 0])], (<class 'dict'>, [['cov_cl', None], ['cls', (<class 'dict'>, [[13, array([8.06555501e-07, 1.05894202e-06, 1.27702410e-06, ...,
       1.33716681e-08, 1.33626402e-08, 1.33536131e-08])], [24, array([1.99272924e-08, 2.77690215e-08, 3.16494253e-08, ...,
       1.10043657e-10, 1.09962855e-10, 1.09882058e-10])], [14, array([8.06555501e-07, 1.05894202e-06, 1.27702410e-06, ...,
       1.33716681e-08, 1.33626402e-08, 1.33536131e-08])], [23, array([8.53298120e-08, 1.16354891e-07, 1.45832485e-07, ...,
       5.63053987e-09, 5.62697388e-09, 5.62340789e-09])]])], ['s1_s2', (<class 'tuple'>, [0, 2])], ['SN', (<class 'dict'>, [[(<class 'tuple'>, ['galaxy', 'galaxy']), array([[[4.93274276e-09, 0.00000000e+00],
        [0.00000000e+00, 8.59388519e

distributed.worker - WARNING -  Compute Failed
Function:  execute_task
args:      ((<function apply at 0x1469e41fe1f0>, <bound method Covariance_utils.xi_cov of <skylens.cov_utils.Covariance_utils object at 0x1469c8120d60>>, [(<class 'tuple'>, [1, 1, 0, 1])], (<class 'dict'>, [['cov_cl', None], ['cls', (<class 'dict'>, [[13, array([8.06555501e-07, 1.05894202e-06, 1.27702410e-06, ...,
       1.33716681e-08, 1.33626402e-08, 1.33536131e-08])], [24, array([8.53298120e-08, 1.16354891e-07, 1.45832485e-07, ...,
       5.63053987e-09, 5.62697388e-09, 5.62340789e-09])], [14, array([8.53298120e-08, 1.16354891e-07, 1.45832485e-07, ...,
       5.63053987e-09, 5.62697388e-09, 5.62340789e-09])], [23, array([8.06555501e-07, 1.05894202e-06, 1.27702410e-06, ...,
       1.33716681e-08, 1.33626402e-08, 1.33536131e-08])]])], ['s1_s2', (<class 'tuple'>, [2, -2])], ['SN', (<class 'dict'>, [[(<class 'tuple'>, ['galaxy', 'galaxy']), array([[[4.93274276e-09, 0.00000000e+00],
        [0.00000000e+00, 8.59388519

distributed.worker - WARNING -  Compute Failed
Function:  execute_task
args:      ((<function apply at 0x1469e41fe1f0>, <bound method Covariance_utils.xi_cov of <skylens.cov_utils.Covariance_utils object at 0x1469c8120d60>>, [(<class 'tuple'>, [1, 1, 1, 0])], (<class 'dict'>, [['cov_cl', None], ['cls', (<class 'dict'>, [[13, array([8.53298120e-08, 1.16354891e-07, 1.45832485e-07, ...,
       5.63053987e-09, 5.62697388e-09, 5.62340789e-09])], [24, array([1.99272924e-08, 2.77690215e-08, 3.16494253e-08, ...,
       1.10043657e-10, 1.09962855e-10, 1.09882058e-10])], [14, array([1.99272924e-08, 2.77690215e-08, 3.16494253e-08, ...,
       1.10043657e-10, 1.09962855e-10, 1.09882058e-10])], [23, array([8.53298120e-08, 1.16354891e-07, 1.45832485e-07, ...,
       5.63053987e-09, 5.62697388e-09, 5.62340789e-09])]])], ['s1_s2', (<class 'tuple'>, [2, -2])], ['SN', (<class 'dict'>, [[(<class 'tuple'>, ['galaxy', 'galaxy']), array([[[4.93274276e-09, 0.00000000e+00],
        [0.00000000e+00, 8.59388519

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [ ]:
print('Got data and cov')

In [ ]:
kappa0.do_cov=False

In [ ]:
kappa0=Skylens(zs_bins=zs_bin,do_cov=do_cov,bin_cl=bin_cl,l_bins=l_bins,l=l0, zg_bins=zs_bin,
               use_window=use_window,Tri_cov=Tri_cov,
               use_binned_l=use_binned_l,wigner_files=wigner_files,
               SSV_cov=SSV_cov,tidal_SSV_cov=tidal_SSV_cov,f_sky=0.35,
               store_win=store_win,window_lmax=window_lmax,
               sparse_cov=True,corrs=corrs,
               do_xi=do_xi,bin_xi=bin_xi,theta_bins=th_bins,WT=WT,
                use_binned_theta=use_binned_theta,
                nz_PS=10,do_pseudo_cl=do_pseudo_cl,xi_win_approx=True,
                xi_SN_analytical=xi_SN_analytical,power_spectra_kwargs=power_spectra_kwargs,
               Win=kappa0.Win
               )

In [ ]:
params_order=['b1_0','b1_1']#,'Ase9','Om']

priors_max=[2,0]#,10,.5]
priors_min=[0.5,0.5]#,0.1,0.1]

cosmo_fid=kappa0.Ang_PS.PS.cosmo_params
Ang_PS=kappa0.Ang_PS


In [ ]:
zs_bin1=copy.deepcopy(zs_bin)
del_k=['window','window_cl']
for k in del_k:
    if zs_bin1.get(k) is not None:
        del zs_bin1[k]
    for i in np.arange(zs_bin1['n_bins']):
        if zs_bin1[i].get(k) is not None:
            del zs_bin1[i][k]

In [ ]:
def get_priors(params):#assume flat priors for now
    x=np.all(params>priors_max,axis=1)
    x*=np.all(params<priors_max,axis=1)
    p=np.zeros(len(params))
    p[x]=-np.inf
    return p

In [ ]:
def assign_zparams(zbins={},p_name='',p_value=None):
    pp=p_name.split('_')
    p_n=pp[0]
    bin_indx=np.int(pp[1])
    zbins[bin_indx][p_n]=p_value
    return zbins

In [ ]:
def get_model(params,data,kappa0,z_bins,log_prior,indx):
#     t1=time.time()
#     print('get_model',params,log_prior)
    if not np.isfinite(log_prior):
        return np.zeros_like(data)
#     kappa=copy.deepcopy(kappa0)
    kappa=kappa0
    cosmo_params=copy.deepcopy(cosmo_fid)
    Ang_PS=copy.deepcopy(kappa.Ang_PS)
    zbins=copy.deepcopy(z_bins)
#     t2=time.time()
    i=0
    for p in params_order:
        if cosmo_params.get(p) is not None:
            cosmo_params[p]=params[i]
        else:
            zbins=assign_zparams(zbins=zbins,p_name=p,p_value=params[i])
        i+=1
    zbins={'galaxy':zbins,'shear':zbins}
#     t3=time.time()
    model=kappa.tomo_short(cosmo_params=cosmo_params,z_bins=zbins,Ang_PS=Ang_PS)
#     print('get_model',indx,time.time()-t1,t2-t1,t3-t2)
    return model

In [ ]:
def chi_sq(params,data,cov_inv,kappa0,z_bins):
    t1=time.time()
    params=np.atleast_2d(params)
    log_priors=get_priors(params)
#     print('params shape',params.shape)
    n_params=len(params)
#     models=client.map(get_model,params)
#     print(models)
#     models=client.gather(models)
#     print(models.shape)
    models={}
    for i in np.arange(n_params):
        models[i]=delayed(get_model)(params[i],data,kappa0,z_bins,log_priors[i],i)
    models=client.compute(models).result()
    models=np.array(list(models.values())) #fixme: check to ensure ordering
    loss=data-models
    chisq=np.zeros(n_params)-np.inf
    for i in np.arange(n_params):
      chisq[i]=-0.5*loss[i]@cov_inv@loss[i].T
#     chisq=-0.5*loss@(cov_inv@loss.T)
#     chisq=np.diag(chisq) #fixme: check to ensure ordering
    chisq+=log_priors
    print('chisq',time.time()-t1)
    return chisq

In [ ]:
def ini_walkers():
    ndim=len(params_order)
    p0=np.zeros(ndim)
    p0f=np.zeros(ndim)
    i=0
    for p in params_order:
        if cosmo_fid.get(p) is not None:
            p0[i]=cosmo_fid[p]
            p0f=p0[i]*.5
        else:
            pp=p.split('_')
            p_n=pp[0]
            bin_indx=np.int(pp[1])
#             print(bin_indx,p_n,zs_bin1[bin_indx].keys())
            p0[i]=zs_bin1[bin_indx][p_n]
            p0f=.2
        i+=1
    return p0,p0f,ndim

In [ ]:
nwalkers=100
nsteps_burn=1
thin=1
ncpu=25
nsteps=10

In [ ]:
def sample_params(fname=''):
    p00,p0_f,ndim=ini_walkers()
    p0 = np.random.uniform(-1,1,ndim * nwalkers).reshape((nwalkers, ndim))*p0_f*p00 + p00

    outp={}
    sampler = emcee.EnsembleSampler(nwalkers, ndim,chi_sq,threads=ncpu,a=2,vectorize=True,args=(data,cov_inv,kappa0,zs_bin1))
                                                                #a=2 default, 5 ok

    t1=time.time()

    pos, prob, state = sampler.run_mcmc(p0, nsteps_burn,store=False)
    print('done burn in '+str(time.time()-t1)+'  '+str((time.time()-t1)/3600.)+'  '+
    str(np.mean(sampler.acceptance_fraction)))

    sampler.reset()

    step_size=nsteps
#     if step_size%thin!=0 or step_size==0:
#         step_size=max(1,int(step_size/thin)*thin+thin)
        
#         step_size=nsteps/10 #30
#         if step_size%thin!=0 or step_size==0:
#             step_size=max(1,int(step_size/thin)*thin+thin)
#         #print 'step-size',step_size,thin
#         steps_taken=0
#     while steps_taken<nsteps:
    pos, prob, state =sampler.run_mcmc(pos, step_size,thin=thin)

    outp['chain']=sampler.flatchain
    outp['p0']=p00
    outp['params']=params_order
    outp['ln_prob']=sampler.lnprobability.flatten()
    outp['acceptance_fraction']=np.mean(sampler.acceptance_fraction)
    outp['pos']=pos
    outp['prob']=prob
    outp['nsteps']=nsteps
    outp['nwalkers']=nwalkers
    outp['burnin']=nsteps_burn
    outp['thin']=thin
    outp['data_corrected']=self.data_corrected
    outp['time']=time.time()-t1

#     with open(fname, 'wb') as f:
#         pickle.dump(outp,f)
#     f.close()
    print('Done steps '+str(steps_taken)+ ' acceptance fraction ' +str(outp['acceptance_fraction'])+'  '
    'time'+str(time.time()-t1)+str((time.time()-t1)/3600.))
    return outp

In [ ]:
kappa0.Ang_PS.reset()
# kappa0.Ang_PS.angular_power_z()

In [ ]:
outp=sample_params()

In [ ]:
outp['l0']=l0
outp['l_bins']=l_bins
outp['do_xi']=do_xi
outp['do_pseudo_cl']=do_pseudo_cl
outp['use_binned_l']=use_binned_l
outp['use_binned_theta']=use_binned_theta
outp['data']=data
outp['zbins']=zs_bin1
outp['cov_inv']=cov_inv
outp['params_order']=params_order


In [ ]:
file_home='/verafs/scratch/phy200040p/sukhdeep/physics2/skylens/tests/imaster/'
if do_xi:
    fname_out='xi_{nz}_bl{bl}_bth{bth}_nw{nw}_ns{ns}.pkl'.format(nz=zs_bin1['nbins'],bl=np.int(use_binned_l),bth=np.int(use_binned_theta),ns=nsteps,nw=nwalkers)
if do_pseudo_cl:
    fname_out='pcl_{nz}_bl_bth.pkl'.format(nz=zs_bin1['nbins'],bl=np.int(use_binned_l),bth=np.int(use_binned_theta))
fname_out=file_home+fname_out
with open(fname_out, 'wb') as f:
    pickle.dump(outp,f)

In [ ]:
from skylens import *
PS=Power_Spectra()

In [ ]:
z1=np.linspace(.1,1,10)
%time PS.pk_func(z=z1)

In [ ]:
z1=np.linspace(.1,1,100)
%time PS.pk_func(z=z1)

In [ ]:

from tensorflow_probability.substrates import jax as tfp
import corner

In [ ]:

def model_func(params,x):
    return params[0]+params[1]*x

def target_log_prob_fn(params0,params1):
    model=model_func([params0,params1],x)
    loss=data-model
    chisq=loss@cov_inv@loss
    return -0.5*chisq 

In [ ]:
Nsamples = 6000 # final number of samples
Nburn = 200  # number of tuning samples


In [ ]:
x=np.linspace(1,10,10)
data=1+x*2
cov=np.eye(10)
cov_inv=np.linalg.inv(cov)

In [ ]:
loss=data
loss@cov_inv@loss

In [ ]:
#https://rlhick.people.wm.edu/posts/custom-likes-tensorflow.html
#http://mattpitkin.github.io/samplers-demo/pages/tensorflow-probability/

# https://rlhick.people.wm.edu/posts/faster-tensorflow.html

#https://www.tensorflow.org/probability/examples/TensorFlow_Probability_on_JAX

In [ ]:
from jax import random
key = random.PRNGKey(0)

In [ ]:
random.uniform(minval=0, maxval=1,key=key)

In [ ]:
@jit
def do_sampling():
    # set initial state (random draw from prior)
    qc=random.uniform(minval=0, maxval=1,key=key)
    qm=random.normal(key=key)
    print('do sampling',qc,qm)
#     qc = tf.random.uniform([], minval=0, maxval=1, name="init_c")
#     qm = tf.random.normal([], stddev=1, mean=1, name="init_m")

#     hmc_kernel = tfp.mcmc.NoUTurnSampler( #https://www.tensorflow.org/probability/api_docs/python/tfp/mcmc/NoUTurnSampler
    hmc_kernel = tfp.mcmc.HamiltonianMonteCarlo(
        target_log_prob_fn=target_log_prob_fn,
        step_size=0.01,num_leapfrog_steps=3
    )

    return tfp.mcmc.sample_chain(
        num_results=Nsamples,
        current_state=[qm, qc],
        kernel=hmc_kernel,
        num_burnin_steps=Nburn,
        parallel_iterations=100,seed=key
    )

states, kernel_results = do_sampling()
# extract the samples
cs, ms = states

# convert output states to numpy arrays
postsamples = np.vstack((ms, cs)).T


In [ ]:
f=corner.corner(postsamples, labels=[r"$m$", r"$c$"],figsize=(20,20),whspace=0.01) #truths=[m, c])
f.subplots_adjust(hspace=0, wspace=0)